In [ ]:
import sys
 
sys.path.append('..')
from piece_v2 import piece_v2

piece_v2.start(should_send_to_score=False)

In [ ]:
from enum import StrEnum
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
import random
from soundmining_tools.generative import *
import math
from soundmining_tools.modular.instrument import AddAction
from soundmining_tools.sequencer import SequenceNote
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
import ipywidgets as widgets

SOUNDPATH = (
    "/Users/danielstahl/Documents/Music/Pieces/Concrete Music/Concrete Music 14/sounds/Concrete Music 14_v3_sounds"
)

Sounds = StrEnum(
    "Sounds",
    [
        "CARDIOID_HIT_1",
        "OMNI_BIG_RATTLE_1",
        "OMNI_BIG_RATTLE_2",
        "OMNI_HIT_1",
        "OMNI_PAN_HIT_1",
        "OMNI_PAN_HIT_2",
        "OMNI_PAN_SCRATH_1",
        "OMNI_PAN_SCRATH_2",
        "OMNI_SMALL_RATTLE_1",
        "OMNI_SMALL_RATTLE_2",
    ],
)

piece_v2.reset()

(
    piece_v2.synth_player
        .add_sound(Sounds.CARDIOID_HIT_1, f"{SOUNDPATH}/Cardioid Hit 1.flac", 0.334, 0.555) # 70, 117, 363, 562, 750, 938, 1137, 1313, 1523, 1723, 1863, 
        .add_sound(Sounds.OMNI_BIG_RATTLE_1, f"{SOUNDPATH}/Omni Big Rattle 1.flac", 0.0, 2.224)
        .add_sound(Sounds.OMNI_BIG_RATTLE_2, f"{SOUNDPATH}/Omni Big Rattle 2.flac", 0.0, 2.615)
        .add_sound(Sounds.OMNI_HIT_1, f"{SOUNDPATH}/Omni Hit 1.flac", 0.202, 0.450)
        .add_sound(Sounds.OMNI_PAN_HIT_1, f"{SOUNDPATH}/Omni Pan Hit 1.flac", 0.425, 0.997) # (190, 421), 614, 704, 1054, 1148, 1895, 2697. 2719, 2905, 3139, 3680, 4522, 5063
        .add_sound(Sounds.OMNI_PAN_HIT_2, f"{SOUNDPATH}/Omni Pan Hit 2.flac", 0.413, 0.924)
        .add_sound(Sounds.OMNI_PAN_SCRATH_1, f"{SOUNDPATH}/Omni Pan Scratch 1.flac", 0.096, 1.551)
        .add_sound(Sounds.OMNI_PAN_SCRATH_2, f"{SOUNDPATH}/Omni Pan Scratch 2.flac", 0.043, 1.228)
        .add_sound(Sounds.OMNI_SMALL_RATTLE_1, f"{SOUNDPATH}/Omni Small Rattle 1.flac", 0.062, 2.074)
        .add_sound(Sounds.OMNI_SMALL_RATTLE_2, f"{SOUNDPATH}/Omni Small Rattle 2.flac", 0.015, 2.126)
        .start()
)

SoundGroup = StrEnum("SoundGroup", ["CARDIOID_HIT", "OMNI_BIG_RATTLE", "OMNI_HIT", "OMNI_PAN_HIT", "OMNI_PAN_SCRATCH", "OMNI_SMALL_RATTLE"])

sound_groups = {
    SoundGroup.CARDIOID_HIT: [Sounds.CARDIOID_HIT_1],
    SoundGroup.OMNI_BIG_RATTLE: [Sounds.OMNI_BIG_RATTLE_1, Sounds.OMNI_BIG_RATTLE_2],
    SoundGroup.OMNI_HIT: [Sounds.OMNI_HIT_1], 
    SoundGroup.OMNI_PAN_HIT: [Sounds.OMNI_PAN_HIT_1, Sounds.OMNI_PAN_HIT_2],
    SoundGroup.OMNI_PAN_SCRATCH: [Sounds.OMNI_PAN_SCRATH_1, Sounds.OMNI_PAN_SCRATH_2],
    SoundGroup.OMNI_SMALL_RATTLE: [Sounds.OMNI_SMALL_RATTLE_1, Sounds.OMNI_SMALL_RATTLE_2]
}

static_control = piece_v2.instruments.static_control
sine_control = piece_v2.instruments.sine_control
perc_control = piece_v2.instruments.perc_control
line_control = piece_v2.instruments.line_control
signal_sum = piece_v2.instruments.signal_sum
signal_multiply = piece_v2.instruments.signal_multiply
three_block_control = piece_v2.instruments.three_block_control

SoundType = StrEnum("SoundType", ["LOW", "MIDDLE", "HIGH"])

sound_type_pitch_ranges = {
    SoundType.LOW: (30, 100),
    SoundType.MIDDLE: (200, 700),
    SoundType.HIGH: (1000, 10000)
}

sound_group_amp_factor = {
    SoundGroup.CARDIOID_HIT: 1,
    SoundGroup.OMNI_BIG_RATTLE: 2,
    SoundGroup.OMNI_HIT: 1, 
    SoundGroup.OMNI_PAN_HIT: 1,
    SoundGroup.OMNI_PAN_SCRATCH: 2,
    SoundGroup.OMNI_SMALL_RATTLE: 2
}

sound_type_amp_factor = {
    SoundType.LOW: 20,
    SoundType.MIDDLE: 10,
    SoundType.HIGH: 10
}

sound_type_pan_points = {
    SoundType.LOW: [(-0.25, 0.25)],
    SoundType.MIDDLE: [(-0.75, -0.25), (0.25, 0.75)],
    SoundType.HIGH:  [(-0.99, -0.75), (0.75, 0.99)]
}

PanType = StrEnum("AmpType", ["LEFT_WIDE", "RIGHT_WIDE", "WIDE", "LEFT_MIDDLE", "RIGHT_MIDDLE", "MIDDLE", "NARROW"])

pan_type_pan_points = {
    PanType.LEFT_WIDE: [(-0.99, -0.75)],
    PanType.RIGHT_WIDE: [(0.75, 0.99)] ,
    PanType.WIDE: [(-0.99, -0.75), (0.75, 0.99)],
    PanType.LEFT_MIDDLE: [(-0.75, -0.25)],
    PanType.RIGHT_MIDDLE: [(0.25, 0.75)],
    PanType.MIDDLE: [(-0.75, -0.25), (0.25, 0.75)],
    PanType.NARROW: [(-0.25, 0.25)]
}

def play_sound(start_time: float, sound_group: SoundGroup, sound_type: SoundType, pan_ranges: list[tuple[float, float]], filter_range: tuple[float, float], track: str) -> SequenceNote:
    sound = random.choice(sound_groups[sound_group])

    low_filter, hight_filter = filter_range

    low_pitch, high_pitch = sound_type_pitch_ranges[sound_type]
    pitch = random_range(low_pitch, high_pitch)
    bw = random_range(100, 300)
    static_amp_factor = sound_type_amp_factor[sound_type] * sound_group_amp_factor[sound_group] * random_range(0.15, 0.85)    
    pan = pan_point(pan_ranges)
    rq = bw / pitch
    amp_factor = (1 / math.sqrt(rq)) * static_amp_factor    
    (
        piece_v2.synth_player.note()
        .sound_mono(sound, 1.0, static_control(2.0))
        .mono_band_pass_filter(static_control(pitch), static_control(rq))
        .mono_low_pass_filter(static_control(hight_filter))
        .mono_high_pass_filter(static_control(low_filter))
        .mono_volume(static_control(amp_factor))
        .pan(static_control(pan))
        .play(start_time=start_time)
    )
    duration = piece_v2.synth_player.get_sound(sound).duration(1.0)
    return SequenceNote(start=start_time, track=track, duration=duration, freq=pitch)

def play_pad(start_time: float, duration: float, freqs: list[float], filter_range: tuple[float, float], pan_ranges: list[tuple[float, float]], ring_times: tuple[float, float]) -> list[SequenceNote]:
    low_filter, hight_filter = filter_range

    nr_freqs = len(freqs)
    amps = [random_range(0.05, 0.15) for _ in range(nr_freqs)]    
    ring_times_lower, ring_times_upper = ring_times
    _ring_times = [random_range(ring_times_lower, ring_times_upper) for _ in range(nr_freqs)]
    pan = pan_point(pan_ranges)
    (
        piece_v2.synth_player.note()
            .white_noise(sine_control(0, 0.2))
            .bank_of_resonators(freqs, amps, _ring_times)
            .mono_low_pass_filter(static_control(hight_filter))
            .mono_high_pass_filter(static_control(low_filter))
            .pan(static_control(pan))
            .play(start_time=start_time, duration=duration)
    )

TimeCloudRanges = StrEnum("TimeCloudRanges", ["SHORT", "MIDDLE", "LONG"])

time_cloud_ranges = {
    TimeCloudRanges.SHORT: (0.02, 0.2),
    TimeCloudRanges.MIDDLE: (0.2, 0.5),
    TimeCloudRanges.LONG: (0.5, 0.8),
}


def mvmnt1_high_hit_cloud_1(start_time: float, duration: float) -> list[SequenceNote]:
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5}
    }, SoundGroup.CARDIOID_HIT)

    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.8, PanType.MIDDLE: 0.2},
        PanType.MIDDLE: {PanType.WIDE: 0.9, PanType.MIDDLE: 0.1}
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.6, TimeCloudRanges.MIDDLE: 0.3, TimeCloudRanges.LONG: 0.1},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.7, TimeCloudRanges.MIDDLE: 0.2, TimeCloudRanges.LONG: 0.1},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.7, TimeCloudRanges.MIDDLE: 0.3, TimeCloudRanges.LONG: 0.0}
    }, TimeCloudRanges.SHORT)

    time = start_time
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = SoundType.HIGH
    
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[SoundType.HIGH], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)


def mvmnt1_middle_hit_cloud_2(start_time: float, duration: float) -> list[SequenceNote]:
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5}
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.8, SoundType.HIGH: 0.2},
        SoundType.HIGH: {SoundType.MIDDLE: 0.95, SoundType.HIGH: 0.05},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.MIDDLE: {PanType.MIDDLE: 0.8, PanType.WIDE: 0.1, PanType.NARROW: 0.1},
        PanType.WIDE: {PanType.MIDDLE: 0.9, PanType.WIDE: 0.0, PanType.NARROW: 0.1},
        PanType.NARROW: {PanType.MIDDLE: 0.9, PanType.WIDE: 0.1, PanType.NARROW: 0.0}
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.2, TimeCloudRanges.MIDDLE: 0.6, TimeCloudRanges.LONG: 0.2},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.3, TimeCloudRanges.MIDDLE: 0.5, TimeCloudRanges.LONG: 0.2},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.4, TimeCloudRanges.MIDDLE: 0.6, TimeCloudRanges.LONG: 0.0}
    }, TimeCloudRanges.MIDDLE)

    time = start_time
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = sound_types_chain.next()
    
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)


def mvmnt1_middle_cloud_4(start_time: float, duration: float) -> list[SequenceNote]:
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_BIG_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0},
        SoundGroup.OMNI_SMALL_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0}
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.7, SoundType.HIGH: 0.3},
        SoundType.HIGH: {SoundType.MIDDLE: 0.9, SoundType.HIGH: 0.1},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.3, PanType.MIDDLE: 0.7},
        PanType.MIDDLE: {PanType.WIDE: 0.2, PanType.MIDDLE: 0.8}
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.5, TimeCloudRanges.MIDDLE: 0.1, TimeCloudRanges.LONG: 0.5},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.4, TimeCloudRanges.MIDDLE: 0.0, TimeCloudRanges.LONG: 0.6},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.6, TimeCloudRanges.MIDDLE: 0.1, TimeCloudRanges.LONG: 0.2}
    }, TimeCloudRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = sound_types_chain.next()
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)


def mvmnt1_low_cloud_6(start_time: float, duration: float) -> list[SequenceNote]:
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_BIG_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0},
        SoundGroup.OMNI_SMALL_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0}
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.4, SoundType.HIGH: 0.1, SoundType.LOW: 0.5},
        SoundType.HIGH: {SoundType.MIDDLE: 0.4, SoundType.HIGH: 0.0, SoundType.LOW: 0.6},
        SoundType.LOW: {SoundType.MIDDLE: 0.5, SoundType.HIGH: 0.1, SoundType.LOW: 0.4},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.0, PanType.MIDDLE: 0.4, PanType.NARROW: 0.6},
        PanType.MIDDLE: {PanType.WIDE: 0.1, PanType.MIDDLE: 0.4, PanType.NARROW: 0.5},
        PanType.NARROW: {PanType.WIDE: 0.1, PanType.MIDDLE: 0.5, PanType.NARROW: 0.4},
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.5},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.5},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.5, TimeCloudRanges.LONG: 0.4}
    }, TimeCloudRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = sound_types_chain.next()
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)
    

PadDuration = StrEnum("PadDuration", ["SHORT", "MIDDLE", "LONG"])

pad_durations = {
    PadDuration.SHORT: (3, 5),
    PadDuration.MIDDLE: (8, 13),
    PadDuration.LONG: (13, 21)
}

PadTime = StrEnum("PadTime", ["EARLY", "MIDDLE", "LATE"])

pad_times = {
    PadTime.EARLY: (0.15, 0.35),
    PadTime.MIDDLE: (0.35, 0.65),
    PadTime.LATE: (0.75, 0.95)
}

PadType = StrEnum("PadType", ["NOISE", "PITCH"])

pad_type_ring_times = {
    (PadType.PITCH, SoundType.LOW): (0.05, 0.2),
    (PadType.PITCH, SoundType.MIDDLE): (0.02, 0.15),
    (PadType.PITCH, SoundType.HIGH): (0.02, 0.1),
    (PadType.NOISE, SoundType.LOW): (0.02, 0.04),
    (PadType.NOISE, SoundType.MIDDLE): (0.01, 0.03),
    (PadType.NOISE, SoundType.HIGH): (0.01, 0.02),
}

hit_pitches = {
    SoundType.LOW: [70, 117, 363, 562],
    SoundType.MIDDLE: [562, 750, 938],
    SoundType.HIGH: [938, 1137, 1313, 1523, 1723, 1863]
}

def mvmnt1_middle_pad_part_5(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.3, PadTime.MIDDLE: 0.6, PadTime.LATE: 0.1},
            PadTime.MIDDLE: {PadTime.EARLY: 0.3, PadTime.MIDDLE: 0.5, PadTime.LATE: 0.2},
            PadTime.LATE: {PadTime.EARLY: 0.4, PadTime.MIDDLE: 0.6, PadTime.LATE: 0.0}
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.MIDDLE: {PadDuration.MIDDLE: 0.4, PadDuration.LONG: 0.6},
            PadDuration.LONG: {PadDuration.MIDDLE: 0.6, PadDuration.LONG: 0.4},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.LOW: {SoundType.MIDDLE: 0.5, SoundType.HIGH: 0.2, SoundType.LOW: 0.3},
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.2, SoundType.HIGH: 0.2, SoundType.LOW: 0.6},
        SoundType.HIGH: {SoundType.MIDDLE: 0.5, SoundType.HIGH: 0.1, SoundType.LOW: 0.4}
    }, SoundType.MIDDLE)
    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.1, PanType.MIDDLE: 0.6, PanType.NARROW: 0.3},
        PanType.MIDDLE: {PanType.WIDE: 0.2, PanType.MIDDLE: 0.4, PanType.NARROW: 0.4},
        PanType.NARROW: {PanType.WIDE: 0.4, PanType.MIDDLE: 0.5, PanType.NARROW: 0.1}
    }, PanType.WIDE)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.3, PadType.PITCH: 0.7},
        PadType.PITCH: {PadType.NOISE: 0.8, PadType.PITCH: 0.2},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))


def mvmnt1_high_pad_part_3(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.2, PadTime.MIDDLE: 0.8},
            PadTime.MIDDLE: {PadTime.EARLY: 0.7, PadTime.MIDDLE: 0.3},
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.MIDDLE: {PadDuration.MIDDLE: 0.4, PadDuration.LONG: 0.6},
            PadDuration.LONG: {PadDuration.MIDDLE: 0.8, PadDuration.LONG: 0.2},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.3, SoundType.HIGH: 0.7},
        SoundType.HIGH: {SoundType.MIDDLE: 0.7, SoundType.HIGH: 0.3}
    }, SoundType.MIDDLE)
    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3},
        PanType.MIDDLE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3}
    }, PanType.WIDE)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
        PadType.PITCH: {PadType.NOISE: 0.8, PadType.PITCH: 0.2},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))

PulseCloudTimeRanges = StrEnum("PulseCloudTimeRanges", ["SHORT", "MIDDLE", "LONG"])

pulse_cloud_time_ranges = {
    PulseCloudTimeRanges.SHORT: (0.01, 0.05),
    PulseCloudTimeRanges.MIDDLE: (0.05, 0.2),
    PulseCloudTimeRanges.LONG: (0.3, 0.5),
}

def mvmnt2_low_pulse_cloud_part_1(start_time: float, duration: float):
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5},
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.1, SoundType.HIGH: 0.0, SoundType.LOW: 0.9},
        SoundType.HIGH: {SoundType.MIDDLE: 0.1, SoundType.HIGH: 0.0, SoundType.LOW: 0.9},
        SoundType.LOW: {SoundType.MIDDLE: 0.1, SoundType.HIGH: 0.0, SoundType.LOW: 0.9},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.0, PanType.MIDDLE: 0.4, PanType.NARROW: 0.6},
        PanType.MIDDLE: {PanType.WIDE: 0.1, PanType.MIDDLE: 0.4, PanType.NARROW: 0.5},
        PanType.NARROW: {PanType.WIDE: 0.1, PanType.MIDDLE: 0.5, PanType.NARROW: 0.4},
    }, PanType.WIDE)

    pulse_cloud_time_ranges_chain = MarkovChain({
        PulseCloudTimeRanges.SHORT: {TimeCloudRanges.SHORT: 0.4, TimeCloudRanges.MIDDLE: 0.5, TimeCloudRanges.LONG: 0.1},
        PulseCloudTimeRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.5, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.1},
        PulseCloudTimeRanges.LONG: {TimeCloudRanges.SHORT: 0.5, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.0}
    }, PulseCloudTimeRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        group_time = time
        group_type_end = group_time + random_range(0.2, 0.5)
        while group_time < group_type_end:
            sound_group = cloud_sounds_chain.next()
            pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
            sound_type = sound_types_chain.next()
            play_sound(group_time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
            time_range_start, time_range_end = pulse_cloud_time_ranges[pulse_cloud_time_ranges_chain.next()]
            group_time = group_time + random_range(time_range_start, time_range_end)
        time = time + random_range(5, 8)
    

def mvmnt2_high_pulse_cloud_part_2(start_time: float, duration: float):
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5},
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.1, SoundType.HIGH: 0.9, SoundType.LOW: 0.0},
        SoundType.HIGH: {SoundType.MIDDLE: 0.1, SoundType.HIGH: 0.9, SoundType.LOW: 0.0},
        SoundType.LOW: {SoundType.MIDDLE: 0.1, SoundType.HIGH: 0.9, SoundType.LOW: 0.0},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.6, PanType.MIDDLE: 0.3, PanType.NARROW: 0.1},
        PanType.MIDDLE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.2, PanType.NARROW: 0.1},
        PanType.NARROW: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3, PanType.NARROW: 0.0},
    }, PanType.WIDE)

    pulse_cloud_time_ranges_chain = MarkovChain({
        PulseCloudTimeRanges.SHORT: {TimeCloudRanges.SHORT: 0.4, TimeCloudRanges.MIDDLE: 0.5, TimeCloudRanges.LONG: 0.1},
        PulseCloudTimeRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.5, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.1},
        PulseCloudTimeRanges.LONG: {TimeCloudRanges.SHORT: 0.5, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.0}
    }, PulseCloudTimeRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        group_time = time
        group_type_end = group_time + random_range(0.2, 0.3)
        while group_time < group_type_end:
            sound_group = cloud_sounds_chain.next()
            pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
            sound_type = sound_types_chain.next()
            play_sound(group_time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
            time_range_start, time_range_end = pulse_cloud_time_ranges[pulse_cloud_time_ranges_chain.next()]
            group_time = group_time + random_range(time_range_start, time_range_end)
        time = time + random_range(3, 5)

def mvmnt2_middle_pad_part_3(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.2, PadTime.MIDDLE: 0.8},
            PadTime.MIDDLE: {PadTime.EARLY: 0.7, PadTime.MIDDLE: 0.3},
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.MIDDLE: {PadDuration.MIDDLE: 0.4, PadDuration.LONG: 0.6},
            PadDuration.LONG: {PadDuration.MIDDLE: 0.8, PadDuration.LONG: 0.2},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.LOW: {SoundType.LOW: 0.1, SoundType.MIDDLE: 0.8, SoundType.HIGH: 0.1},
        SoundType.MIDDLE: {SoundType.LOW: 0.2, SoundType.MIDDLE: 0.6, SoundType.HIGH: 0.2},
        SoundType.HIGH: {SoundType.LOW: 0.2, SoundType.MIDDLE: 0.8, SoundType.HIGH: 0.0}
    }, SoundType.MIDDLE)
    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3},
        PanType.MIDDLE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3}
    }, PanType.WIDE)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.2, PadType.PITCH: 0.8},
        PadType.PITCH: {PadType.NOISE: 0.8, PadType.PITCH: 0.2},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))


def mvmnt3_middle_cloud_part_1(start_time: float, duration: float):
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.1, SoundGroup.OMNI_HIT: 0.1, SoundGroup.OMNI_BIG_RATTLE: 0.4, SoundGroup.OMNI_SMALL_RATTLE: 0.4},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.1, SoundGroup.OMNI_HIT: 0.1, SoundGroup.OMNI_BIG_RATTLE: 0.4, SoundGroup.OMNI_SMALL_RATTLE: 0.4},
        SoundGroup.OMNI_BIG_RATTLE: {SoundGroup.CARDIOID_HIT: 0.2, SoundGroup.OMNI_HIT: 0.2, SoundGroup.OMNI_BIG_RATTLE: 0.2, SoundGroup.OMNI_SMALL_RATTLE: 0.4},
        SoundGroup.OMNI_SMALL_RATTLE: {SoundGroup.CARDIOID_HIT: 0.2, SoundGroup.OMNI_HIT: 0.2, SoundGroup.OMNI_BIG_RATTLE: 0.4, SoundGroup.OMNI_SMALL_RATTLE: 0.2}
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.LOW: {SoundType.LOW: 0.1, SoundType.MIDDLE: 0.8, SoundType.HIGH: 0.1},
        SoundType.MIDDLE: {SoundType.LOW: 0.2, SoundType.MIDDLE: 0.7, SoundType.HIGH: 0.1},
        SoundType.HIGH: {SoundType.LOW: 0.1, SoundType.MIDDLE: 0.9, SoundType.HIGH: 0.0},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.NARROW: {PanType.NARROW: 0.2, PanType.WIDE: 0.2, PanType.MIDDLE: 0.6},
        PanType.WIDE: {PanType.NARROW: 0.2, PanType.WIDE: 0.1, PanType.MIDDLE: 0.7},
        PanType.MIDDLE: {PanType.NARROW: 0.3, PanType.WIDE: 0.2, PanType.MIDDLE: 0.5}
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.0, TimeCloudRanges.MIDDLE: 0.2, TimeCloudRanges.LONG: 0.8},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.1, TimeCloudRanges.LONG: 0.8},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.2, TimeCloudRanges.LONG: 0.7}
    }, TimeCloudRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = sound_types_chain.next()
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)


def mvmnt3_high_pad_part_2(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.3, PadTime.MIDDLE: 0.7},
            PadTime.MIDDLE: {PadTime.EARLY: 0.5, PadTime.MIDDLE: 0.5},
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.SHORT: {PadDuration.SHORT: 0.1, PadDuration.MIDDLE: 0.9, PadDuration.LONG: 0.0},
            PadDuration.MIDDLE: {PadDuration.SHORT: 0.4, PadDuration.MIDDLE: 0.6, PadDuration.LONG: 0.0},
            PadDuration.LONG: {PadDuration.SHORT: 0.5, PadDuration.MIDDLE: 0.5, PadDuration.LONG: 0.0},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.3, SoundType.HIGH: 0.7},
        SoundType.HIGH: {SoundType.MIDDLE: 0.7, SoundType.HIGH: 0.3}
    }, SoundType.MIDDLE)
    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3},
        PanType.MIDDLE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3}
    }, PanType.WIDE)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
        PadType.PITCH: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))


def mvmnt3_low_pad_part_3(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.3, PadTime.MIDDLE: 0.7},
            PadTime.MIDDLE: {PadTime.EARLY: 0.5, PadTime.MIDDLE: 0.5},
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.SHORT: {PadDuration.SHORT: 0.1, PadDuration.MIDDLE: 0.9, PadDuration.LONG: 0.0},
            PadDuration.MIDDLE: {PadDuration.SHORT: 0.4, PadDuration.MIDDLE: 0.6, PadDuration.LONG: 0.0},
            PadDuration.LONG: {PadDuration.SHORT: 0.5, PadDuration.MIDDLE: 0.5, PadDuration.LONG: 0.0},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.3, SoundType.LOW: 0.7},
        SoundType.LOW: {SoundType.MIDDLE: 0.7, SoundType.LOW: 0.3}
    }, SoundType.LOW)
    pan_ranges_chain = MarkovChain({
        PanType.NARROW: {PanType.NARROW: 0.5, PanType.MIDDLE: 0.5},
        PanType.MIDDLE: {PanType.NARROW: 0.5, PanType.MIDDLE: 0.5}
    }, PanType.NARROW)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
        PadType.PITCH: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))



def mvmnt4_middle_cloud_part_1(start_time: float, duration: float):
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.1, SoundGroup.OMNI_HIT: 0.1, SoundGroup.OMNI_BIG_RATTLE: 0.4, SoundGroup.OMNI_SMALL_RATTLE: 0.4},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.1, SoundGroup.OMNI_HIT: 0.1, SoundGroup.OMNI_BIG_RATTLE: 0.4, SoundGroup.OMNI_SMALL_RATTLE: 0.4},
        SoundGroup.OMNI_BIG_RATTLE: {SoundGroup.CARDIOID_HIT: 0.2, SoundGroup.OMNI_HIT: 0.2, SoundGroup.OMNI_BIG_RATTLE: 0.2, SoundGroup.OMNI_SMALL_RATTLE: 0.4},
        SoundGroup.OMNI_SMALL_RATTLE: {SoundGroup.CARDIOID_HIT: 0.2, SoundGroup.OMNI_HIT: 0.2, SoundGroup.OMNI_BIG_RATTLE: 0.4, SoundGroup.OMNI_SMALL_RATTLE: 0.2}
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.LOW: {SoundType.LOW: 0.1, SoundType.MIDDLE: 0.8, SoundType.HIGH: 0.1},
        SoundType.MIDDLE: {SoundType.LOW: 0.2, SoundType.MIDDLE: 0.7, SoundType.HIGH: 0.1},
        SoundType.HIGH: {SoundType.LOW: 0.1, SoundType.MIDDLE: 0.9, SoundType.HIGH: 0.0},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.NARROW: {PanType.NARROW: 0.2, PanType.WIDE: 0.2, PanType.MIDDLE: 0.6},
        PanType.WIDE: {PanType.NARROW: 0.2, PanType.WIDE: 0.1, PanType.MIDDLE: 0.7},
        PanType.MIDDLE: {PanType.NARROW: 0.3, PanType.WIDE: 0.2, PanType.MIDDLE: 0.5}
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.2, TimeCloudRanges.MIDDLE: 0.2, TimeCloudRanges.LONG: 0.6},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.3, TimeCloudRanges.MIDDLE: 0.1, TimeCloudRanges.LONG: 0.6},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.3, TimeCloudRanges.MIDDLE: 0.2, TimeCloudRanges.LONG: 0.5}
    }, TimeCloudRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = sound_types_chain.next()
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)


def mvmnt4_low_cloud_part2(start_time: float, duration: float) -> list[SequenceNote]:
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_BIG_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0},
        SoundGroup.OMNI_SMALL_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0}
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.3, SoundType.HIGH: 0.0, SoundType.LOW: 0.7},
        SoundType.HIGH: {SoundType.MIDDLE: 0.4, SoundType.HIGH: 0.0, SoundType.LOW: 0.6},
        SoundType.LOW: {SoundType.MIDDLE: 0.5, SoundType.HIGH: 0.0, SoundType.LOW: 0.5},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.0, PanType.MIDDLE: 0.4, PanType.NARROW: 0.6},
        PanType.MIDDLE: {PanType.WIDE: 0.0, PanType.MIDDLE: 0.4, PanType.NARROW: 0.6},
        PanType.NARROW: {PanType.WIDE: 0.0, PanType.MIDDLE: 0.5, PanType.NARROW: 0.5},
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.5},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.5},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.5, TimeCloudRanges.LONG: 0.4}
    }, TimeCloudRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = sound_types_chain.next()
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)


def mvmnt4_high_cloud_part3(start_time: float, duration: float) -> list[SequenceNote]:
    cloud_sounds_chain = MarkovChain({
        SoundGroup.CARDIOID_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_HIT: {SoundGroup.CARDIOID_HIT: 0.4, SoundGroup.OMNI_HIT: 0.4, SoundGroup.OMNI_BIG_RATTLE: 0.1, SoundGroup.OMNI_SMALL_RATTLE: 0.1},
        SoundGroup.OMNI_BIG_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0},
        SoundGroup.OMNI_SMALL_RATTLE: {SoundGroup.CARDIOID_HIT: 0.5, SoundGroup.OMNI_HIT: 0.5, SoundGroup.OMNI_BIG_RATTLE: 0, SoundGroup.OMNI_SMALL_RATTLE: 0}
    }, SoundGroup.CARDIOID_HIT)

    sound_types_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.3, SoundType.HIGH: 0.7, SoundType.LOW: 0.0},
        SoundType.HIGH: {SoundType.MIDDLE: 0.4, SoundType.HIGH: 0.6, SoundType.LOW: 0.0},
        SoundType.LOW: {SoundType.MIDDLE: 0.5, SoundType.HIGH: 0.5, SoundType.LOW: 0.0},
    }, SoundType.MIDDLE)

    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.6, PanType.MIDDLE: 0.4, PanType.NARROW: 0.0},
        PanType.MIDDLE: {PanType.WIDE: 0.6, PanType.MIDDLE: 0.4, PanType.NARROW: 0.0},
        PanType.NARROW: {PanType.WIDE: 0.5, PanType.MIDDLE: 0.5, PanType.NARROW: 0.0},
    }, PanType.WIDE)

    cloud_time_ranges_chain = MarkovChain({
        TimeCloudRanges.SHORT: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.5},
        TimeCloudRanges.MIDDLE: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.4, TimeCloudRanges.LONG: 0.5},
        TimeCloudRanges.LONG: {TimeCloudRanges.SHORT: 0.1, TimeCloudRanges.MIDDLE: 0.5, TimeCloudRanges.LONG: 0.4}
    }, TimeCloudRanges.SHORT)

    time = start_time 
    end_time = start_time + duration
    while time < end_time:
        sound_group = cloud_sounds_chain.next()
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        sound_type = sound_types_chain.next()
        play_sound(time, sound_group, sound_type, pan_ranges, sound_type_pitch_ranges[sound_type], "The track name")
        time_range_start, time_range_end = time_cloud_ranges[cloud_time_ranges_chain.next()]
        time = time + random_range(time_range_start, time_range_end)

        

def mvmnt4_high_pad_part4(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.3, PadTime.MIDDLE: 0.7},
            PadTime.MIDDLE: {PadTime.EARLY: 0.5, PadTime.MIDDLE: 0.5},
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.SHORT: {PadDuration.SHORT: 0.1, PadDuration.MIDDLE: 0.9, PadDuration.LONG: 0.0},
            PadDuration.MIDDLE: {PadDuration.SHORT: 0.4, PadDuration.MIDDLE: 0.6, PadDuration.LONG: 0.0},
            PadDuration.LONG: {PadDuration.SHORT: 0.5, PadDuration.MIDDLE: 0.5, PadDuration.LONG: 0.0},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.3, SoundType.HIGH: 0.7},
        SoundType.HIGH: {SoundType.MIDDLE: 0.7, SoundType.HIGH: 0.3}
    }, SoundType.MIDDLE)
    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3},
        PanType.MIDDLE: {PanType.WIDE: 0.7, PanType.MIDDLE: 0.3}
    }, PanType.WIDE)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
        PadType.PITCH: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))


def mvmnt4_low_pad_part5(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.3, PadTime.MIDDLE: 0.7},
            PadTime.MIDDLE: {PadTime.EARLY: 0.5, PadTime.MIDDLE: 0.5},
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.SHORT: {PadDuration.SHORT: 0.1, PadDuration.MIDDLE: 0.9, PadDuration.LONG: 0.0},
            PadDuration.MIDDLE: {PadDuration.SHORT: 0.4, PadDuration.MIDDLE: 0.6, PadDuration.LONG: 0.0},
            PadDuration.LONG: {PadDuration.SHORT: 0.5, PadDuration.MIDDLE: 0.5, PadDuration.LONG: 0.0},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.MIDDLE: {SoundType.MIDDLE: 0.3, SoundType.LOW: 0.7},
        SoundType.LOW: {SoundType.MIDDLE: 0.7, SoundType.LOW: 0.3}
    }, SoundType.MIDDLE)
    pan_ranges_chain = MarkovChain({
        PanType.NARROW: {PanType.NARROW: 0.7, PanType.MIDDLE: 0.3},
        PanType.MIDDLE: {PanType.NARROW: 0.7, PanType.MIDDLE: 0.3}
    }, PanType.NARROW)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
        PadType.PITCH: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))

def mvmnt4_middle_pad_part6(start_time: float, duration: float):
    pad_times_chain = MarkovChain({
            PadTime.EARLY: {PadTime.EARLY: 0.3, PadTime.MIDDLE: 0.7},
            PadTime.MIDDLE: {PadTime.EARLY: 0.5, PadTime.MIDDLE: 0.5},
        }, PadTime.MIDDLE
    )
    pad_durations_chain = MarkovChain({
            PadDuration.SHORT: {PadDuration.SHORT: 0.1, PadDuration.MIDDLE: 0.9, PadDuration.LONG: 0.0},
            PadDuration.MIDDLE: {PadDuration.SHORT: 0.4, PadDuration.MIDDLE: 0.6, PadDuration.LONG: 0.0},
            PadDuration.LONG: {PadDuration.SHORT: 0.5, PadDuration.MIDDLE: 0.5, PadDuration.LONG: 0.0},
        }, PadDuration.MIDDLE
    )
    hit_pitches_chain = MarkovChain({
        SoundType.HIGH: {SoundType.HIGH: 0.1, SoundType.MIDDLE: 0.7, SoundType.LOW: 0.2},
        SoundType.MIDDLE: {SoundType.HIGH: 0.3, SoundType.MIDDLE: 0.4, SoundType.LOW: 0.3},
        SoundType.LOW: {SoundType.HIGH: 0.2, SoundType.MIDDLE: 0.7, SoundType.LOW: 0.1}
    }, SoundType.MIDDLE)
    pan_ranges_chain = MarkovChain({
        PanType.WIDE: {PanType.WIDE: 0.1, PanType.MIDDLE: 0.7, PanType.NARROW: 0.2},
        PanType.MIDDLE: {PanType.WIDE: 0.3, PanType.MIDDLE: 0.4, PanType.NARROW: 0.3},
        PanType.NARROW: {PanType.WIDE: 0.2, PanType.MIDDLE: 0.7, PanType.NARROW: 0.1}
    }, PanType.NARROW)
    pad_types_chain = MarkovChain({
        PadType.NOISE: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
        PadType.PITCH: {PadType.NOISE: 0.5, PadType.PITCH: 0.5},
    }, PadType.NOISE)
    time = start_time
    end_time = start_time + duration
    while time < end_time:
        pad_duration_lower, pad_duration_upper = pad_durations[pad_durations_chain.next()]
        duration = random_range(pad_duration_lower, pad_duration_upper)
        hit_pitch_type = hit_pitches_chain.next()
        all_pitches = hit_pitches[hit_pitch_type]
        match hit_pitch_type:
            case SoundType.LOW: 
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.MIDDLE:
                freqs = random.choices(all_pitches, k=random_int_range(2, 3))
            case SoundType.HIGH:
                freqs = random.choices(all_pitches, k=random_int_range(3, 5))
        pan_ranges = pan_type_pan_points[pan_ranges_chain.next()]
        pad_type = pad_types_chain.next()
        ring_times = pad_type_ring_times[(pad_type, hit_pitch_type)]
        filter_range = sound_type_pitch_ranges[hit_pitch_type]
        play_pad(time, duration, freqs, filter_range, pan_ranges, ring_times)
        next_time_low, next_time_high = pad_times[pad_times_chain.next()]
        time += (duration * random_range(next_time_low, next_time_high))

def play_mvmt1(start_time: float):
    time = start_time
    mvmnt1_high_hit_cloud_1(time, 34)
    time += (34 * random_range(0.75, 0.85))
    mvmnt1_middle_hit_cloud_2(time, 34)
    time += (34 * random_range(0.75, 0.85))
    mvmnt1_high_pad_part_3(time, 34)
    time += (34 * random_range(0.75, 0.85))
    mvmnt1_middle_cloud_4(time, 34)
    time += (34 * random_range(0.75, 0.85))
    mvmnt1_middle_pad_part_5(time, 34)
    time += (34 * random_range(0.75, 0.85))
    mvmnt1_low_cloud_6(time, 34)

def play_mvnt2(start_time: float):
    time = start_time
    mvmnt2_low_pulse_cloud_part_1(time, 55)
    time += (random_range(1, 2))
    mvmnt2_middle_pad_part_3(time, 55)
    time += (random_range(2, 3))
    mvmnt2_high_pulse_cloud_part_2(time, 55)


def play_mvnt3(start_time: float):
    time = start_time
    mvmnt3_middle_cloud_part_1(time, 55)
    time += (random_range(1, 2))
    mvmnt3_high_pad_part_2(time, 21)
    time += (random_range(21, 34))
    mvmnt3_low_pad_part_3(time, 21)


def play_mvnt4(start_time: float):
    time = start_time    
    mvmnt4_middle_cloud_part_1(time, 34)
    time += (random_range(1, 2))
    mvmnt4_low_cloud_part2(time, 34)
    time += (random_range(1, 2))
    mvmnt4_high_cloud_part3(time, 34)
    time += (random_range(13, 21))
    mvmnt4_high_pad_part4(time, 34)
    time += (random_range(1, 2))
    mvmnt4_low_pad_part5(time, 34)
    time += (random_range(1, 2))
    mvmnt4_middle_pad_part6(time, 34)


class PadHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    # Hit 70, 117, 363, 562, 750, 938, 1137, 1313, 1523, 1723, 1863, 
    def handle_note(self, patch_arguments: PatchArguments) -> None:
        duration_lower, duration_upper = pad_durations[PadDuration.MIDDLE]
        duration = random_range(duration_lower, duration_upper)
        match patch_arguments.note:
            case 0:                
                freqs = random.choices([70, 117, 363, 562], k=random_int_range(2, 3))
                filter_range = sound_type_pitch_ranges[SoundType.LOW]
                pan_ranges = sound_type_pan_points[SoundType.LOW]
                #ring_times = (0.05, 0.2)
                ring_times = (0.02, 0.04)
                play_pad(patch_arguments.start, duration, freqs, filter_range, pan_ranges, ring_times)
            case 1:                
                freqs = random.choices([562, 750, 938], k=random_int_range(2, 3))
                filter_range = sound_type_pitch_ranges[SoundType.MIDDLE]
                pan_ranges = sound_type_pan_points[SoundType.MIDDLE]
                #ring_times = (0.02, 0.15)
                ring_times = (0.01, 0.03)
                play_pad(patch_arguments.start, duration, freqs, filter_range, pan_ranges, ring_times)
            case 2:                
                freqs = random.choices([938, 1137, 1313, 1523, 1723, 1863], k=random_int_range(3, 5))
                filter_range = sound_type_pitch_ranges[SoundType.HIGH]
                pan_ranges = sound_type_pan_points[SoundType.HIGH]
                #ring_times = (0.02, 0.1)
                ring_times = (0.01, 0.02)
                play_pad(patch_arguments.start, duration, freqs, filter_range, pan_ranges, ring_times)
    


class CloudHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.note:
            case 0:
                play_mvmt1(patch_arguments.start)
            case 1:
                play_mvnt2(patch_arguments.start)
            case 2:
                play_mvnt3(patch_arguments.start)    
            case 3:
                play_mvnt4(patch_arguments.start)



class SoundHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.note:
            case 0:
                sound_group = SoundGroup.CARDIOID_HIT
            case 1:
                sound_group = SoundGroup.OMNI_BIG_RATTLE
            case 2:
                sound_group = SoundGroup.OMNI_HIT
            case 3:
                sound_group = SoundGroup.OMNI_PAN_HIT
            case 4:
                sound_group = SoundGroup.OMNI_PAN_SCRATCH
            case 5 | _:
                sound_group = SoundGroup.OMNI_SMALL_RATTLE        

        match patch_arguments.octave:
            case 2:            
                sound_type = None
            case 3:                
                sound_type = SoundType.LOW
                pan_ranges = pan_type_pan_points[PanType.NARROW]
            case 4:                
                sound_type = SoundType.MIDDLE
                pan_ranges = pan_type_pan_points[PanType.MIDDLE]
            case 5:                
                sound_type = SoundType.HIGH
                pan_ranges = pan_type_pan_points[PanType.WIDE]
            
        if sound_type:
            play_sound(patch_arguments.start, sound_group, sound_type, pan_ranges, "The track name")
        else:
            pass

my_handler = CloudHandler(piece_v2.supercollider_client)
piece_v2.receiver.set_note_handler(my_handler)
        

In [ ]:
piece_v2.stop()